# UR5 Simulation forward kinematics
**Author** Eduardo Gomez

Using the Denavit-Hartenberg parameters of the UR5, program the forward kinematics

In [49]:
# Import libraries
import zmqRemoteApi
import numpy as np
from time import sleep
from pathlib import Path
import pandas as pd

## Setup Data

Here we will set up the data for the Denavit-Hartenberg parameters. The data was from: ...

In [50]:
df = pd.read_csv("../data/ur5-kinematics.csv")
print(df.head())

  Kinematics  theta_[rad]   a_[m]   d_[m]  alpha_[rad]
0    Joint_1            0  0.0000  0.1625         1.57
1    Joint_2            0 -0.4250  0.0000         0.00
2    Joint_3            0 -0.3922  0.0000         0.00
3    Joint_4            0  0.0000  0.1333         1.57
4    Joint_5            0  0.0000  0.0997        -1.57


The matriz transformations can be expressed like these: 

$$
H_i^{i-1} = Rot_z(\theta_i)T(0,0,d_i)T(a_i, 0, 0)Rotx(\alpha_i)
$$

Esto se lee como una rotacion $\theta$ en $z$, una translacion $d$ en $z$, translacion $a$ en $x$ y rotacion $\alpha$ en $x$,

Here we'll define some utility functions

In [63]:
from collections.abc import Callable
import numpy as np
from functools import wraps


def deg_args(f: Callable):
    """
    Transforms all the arguments, assuming to be deg, to rads
    """

    @wraps(f)
    def wrapper(*args, **kwargs):
        new_args = [np.deg2rad(arg) for arg in args]
        new_kwargs = {key: np.deg2rad(value) for key, value in kwargs.items()}
        return f(*new_args, **new_kwargs)

    return wrapper


@deg_args
def rotation_3d_deg(yau_z : float = 0, theta_y : float = 0, phi_x : float = 0 ):
    """
    Returns the rotation matrix in 2d, given the angles
    The angles are calculated assuming the axis is the axis of rotation
    """
    R_x = np.array(
        [
            [1, 0, 0],
            [0, np.cos(phi_x), -np.sin(phi_x)],
            [0, np.sin(phi_x), np.cos(phi_x)],
        ]
    )

    R_y = np.array(
        [
            [np.cos(theta_y), 0, np.sin(theta_y)],
            [0, 1, 0],
            [-np.sin(theta_y), 0, np.cos(theta_y)],
        ]
    )

    R_z = np.array(
        [
            [np.cos(yau_z), -np.sin(yau_z), 0],
            [np.sin(yau_z), np.cos(yau_z), 0],
            [0, 0, 1],
        ]
    )

    return R_z @ R_y @ R_x, {"R_z": R_z, "R_y": R_y, "R_x": R_x}


def add_translation_to_rotation_3d(rotation_3d: np.ndarray, p: np.ndarray):
    return np.block([[rotation_3d, np.reshape(p, (3, 1))], [np.zeros((1, 3)), 1]])


def get_homogenous_point_3d(p: np.ndarray):
    return np.block([p, 1])


def mul_homogenous_matrixes(transform_matrices: list[np.ndarray]) -> np.ndarray:
    """
    Multiplies a list of homogeneous matrixes, from right to left
    """
    return reduce(lambda prev, next: next @ prev, transform_matrices[::-1])



In [52]:
# this is the identity matrix for rotation
NO_ROTATION_MATRIX, _ = rotation_3d_deg(0, 0, 0)
NO_TRANSLATION_VEC = np.array([0,0,0])


def get_homogenous_mat_for_joint(theta : float, alpha : float, a : float, d : float):
    """
    Esto se lee como una rotacion $\theta$ en $z$, una translacion $d$ en $z$, translacion $a$ en $x$ y rotacion $\alpha$ en $x$,
    """
    rotation_z, _ = rotation_3d_deg(yau_z=theta)
    rotation_z = add_translation_to_rotation_3d(rotation_z, NO_TRANSLATION_VEC)
    rotation_x, _ = rotation_3d_deg(phi_x=alpha)
    rotation_x = add_translation_to_rotation_3d(rotation_x, NO_TRANSLATION_VEC)
    
    
    translation_x = add_translation_to_rotation_3d(NO_ROTATION_MATRIX, np.array([a, 0, 0]))
    translation_z = add_translation_to_rotation_3d(NO_ROTATION_MATRIX, np.array([0, 0, d]))
    
    H = rotation_z @ translation_z @ translation_x @ rotation_x
    return H


In [53]:
H = get_homogenous_mat_for_joint(0, 10, 14, 50)
print(H)

[[ 1.          0.          0.         14.        ]
 [ 0.          0.98480775 -0.17364818  0.        ]
 [ 0.          0.17364818  0.98480775 50.        ]
 [ 0.          0.          0.          1.        ]]


In [54]:
print(df)

  Kinematics  theta_[rad]   a_[m]   d_[m]  alpha_[rad]
0    Joint_1            0  0.0000  0.1625         1.57
1    Joint_2            0 -0.4250  0.0000         0.00
2    Joint_3            0 -0.3922  0.0000         0.00
3    Joint_4            0  0.0000  0.1333         1.57
4    Joint_5            0  0.0000  0.0997        -1.57
5    Joint_6            0  0.0000  0.0996         0.00


In [64]:
from functools import reduce


H_mats = [
    get_homogenous_mat_for_joint(theta=row["theta_[rad]"], a=row["a_[m]"], alpha=row["d_[m]"], d=row["alpha_[rad]"])
    for _, row in df.iterrows()
 ]

H_total = mul_homogenous_matrixes(H_mats)

print(H_total)





[[ 1.          0.          0.         -0.8172    ]
 [ 0.          0.99996267 -0.00864102  0.00365261]
 [ 0.          0.00864102  0.99996267  1.57001461]
 [ 0.          0.          0.          1.        ]]


## Setup simulation

To set up the simulation, open the `./sim/ur5-simulation.ttt` file. Then run the following code: 

> If there is an error, either the file is not opened or another script is controlling it

In [65]:
# Create a client to get connected to the zmqremoteApi sernver from CoppeliaSim
client = zmqRemoteApi.RemoteAPIClient()

# Get the remote object 'sim'
sim = client.getObject('sim')


Here, we set up the id for controlling the robot

In [66]:
UR5_sim_obj_id = "/UR5"
base_id = sim.getObject(UR5_sim_obj_id)

# the last joint is the rotor for the end effector
# does not affect the final position
no_joints = 6
joint_sim_names = [None for _ in range(6)]

joint_sim_names[0] = f"{UR5_sim_obj_id}/joint"

for i in range(1,len(joint_sim_names)):
    link_joint_str = "/link/joint" * i
    joint_sim_names[i] = f"{UR5_sim_obj_id}{link_joint_str}"

connection_name = f"{UR5_sim_obj_id}/connection"

In [67]:
# Assert that names have id. Error is thrown if id doesnt exists
joint_id = [
    sim.getObject(name) for name in joint_sim_names
]
connection_id = sim.getObject(connection_name)

In [68]:
def reset_arm():
    """
    Resets the arm position to 0 deg
    """
    for j in joint_id:
        sim.setJointTargetPosition(j, np.deg2rad(0))

In [ ]:
Here we start the simulation

In [69]:
sim.startSimulation()

1

In [70]:
reset_arm()

In [71]:

print(sim.getObjectPosition(connection_id, sim.handle_world))
angles = [90, -45, 80, -40, 100, 80]
angles = [np.deg2rad(a) for a in angles]
assert len(angles) == no_joints

for a, j in zip(angles, joint_id):
    sim.setJointTargetPosition(j, a)

sleep(1)
print("Connection final position")
print(sim.getObjectPosition(connection_id, sim.handle_world))



[-0.38503810662955174, 0.22499981038377784, 1.0011865280398988]
Connection final position
[-0.04242355723547914, 0.12796914759588116, 0.7989468163579071]


Here we stop the simulation

In [11]:
sim.stopSimulation()

1